# This tutorial is running on Geforce GTX 1080Ti 12GB

## Load Basic Library

In [1]:
import math
import numpy as np
import tensorflow as tf

## Set Environment and Parameters

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # use second GPU

seed = 2 # random seed
model_dir = "model_test/" # folder for saving model and log
resize_shape = (128, 128) # resized image size
NUM_LABELS = 12 # number of labels
BATCH_SIZE = 256 # number of images in one batch
EPOCHS = 10
SAVE_SUMMARY_STEPS = 100 # save summary to tensorboard - one step means one batch
NUM_GPUS = 1 # number of GPU

## Load Data
See Analyze.ipynb  
dataset from: https://www.kaggle.com/c/plant-seedlings-classification

In [3]:
import pickle

pickle_file = 'train_val10_{1}*{1}_seed{0}.pickle'.format(seed, resize_shape[0])

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    train_dataset = train_dataset.astype("float32")
    valid_dataset = valid_dataset.astype("float32")
    train_dataset /= 255
    valid_dataset /= 255
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)

Training set (4630, 128, 128, 3) (4630,)
Validation set (120, 128, 128, 3) (120,)


## Label Encoder

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_labels)
valid_labels = encoder.transform(valid_labels)
encoder.classes_

array(['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed',
       'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize',
       'Scentless Mayweed', 'Shepherds Purse',
       'Small-flowered Cranesbill', 'Sugar beet'], dtype='<U25')

## Image Augmentation (Keras)

In [5]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(train_dataset)

# Fit the model on the batches generated by datagen.flow().
generator = datagen.flow(train_dataset, train_labels,
                         batch_size=1, seed=seed)

## Feed Training Data to Model (Keras)

In [6]:
def train_input_fn_keras(generator):
    gen_fn = lambda: generator
    
    dataset = tf.data.Dataset.from_generator(gen_fn, (tf.float32, tf.int64))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.repeat(EPOCHS)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(None)
    
    iterator = dataset.make_one_shot_iterator()
    
    features, labels = iterator.get_next()
    features = tf.reshape(features, [-1, 128, 128, 3])
    print("output feature:", features.shape)
    
    tf.summary.image("images", features)
    
    return features, labels

## Feed Training Data to Model (Original Version)
1. original version can not do image aumentation by `tf.keras.preprocessing.image`, because `dataset` input is **Tensor** type, however `tf.keras.preprocessing.image` input is **numpy** type  
    * Solution 1  - Running time: 181.727 sec per 100 steps (batch_size=256)
    ```python
    def make_generator(images, labels):

        def _generator():
            for image, label in zip(images, labels):
                image = tf.keras.preprocessing.image.random_rotation(np.array(image), 30, row_axis=0, col_axis=1, channel_axis=2)
                image = tf.keras.preprocessing.image.random_shift(image, 0.1, 0.1, row_axis=0, col_axis=1, channel_axis=2)
                yield image, label

        return _generator
    ```
    * Solution 2 - Running time: 105.140 sec per 100 steps (batch_size=256)  
    use `tf.keras.preprocessing.image.ImageDataGenerator`  
    Reference: http://www.scsk.jp/product/oss/tec_guide/tensorflow_keras/1_tensorflow_keras3_3.html
2. `tf.data.Dataset.prefetch` for speed optimization. Pre fetch and load data when GPU is running, i.e. CPU is always running  
    Set **`None`** for tensorflow to choose how many data to pre-load for opitimal speed

In [7]:
def train_input_fn(features, labels):
    
    def make_generator(images, labels):

        def _generator():
            for image, label in zip(images, labels):
                yield image, label

        return _generator
    
    def _image_augmentation(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        return image, label
    
    dataset = tf.data.Dataset.from_generator(make_generator(features, labels), (tf.float32, tf.int64))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.repeat(EPOCHS)
    dataset = dataset.map(_image_augmentation)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(None)
    
    iterator = dataset.make_one_shot_iterator()
    
    features, labels = iterator.get_next()
    features = tf.reshape(features, [-1, 128, 128, 3])
    print("output feature:", features.shape)
    
    tf.summary.image("images", features)
    
    return features, labels

## Feed Evaluation Data to Model

In [8]:
def eval_input_fn(features, labels):
    
    def make_generator(images, labels):

        def _generator():
            for image, label in zip(images, labels):
                yield image, label

        return _generator
    
    dataset = tf.data.Dataset.from_generator(make_generator(features, labels), (tf.float32, tf.int64))
    dataset = dataset.batch(BATCH_SIZE)
    
    iterator = dataset.make_one_shot_iterator()
    
    features, labels = iterator.get_next()
    features = tf.reshape(features, [-1, 128, 128, 3])
    
    return features, labels

## Model Structure
**Remember: Do batch normalization in training mode, but not in evaluation and prediction mode**  
This model Structure based on VGG net

In [9]:
def model_structure(features, training):
    
    # Block 1
    features = tf.layers.conv2d(features, filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block1_conv1")
    features = tf.layers.batch_normalization(features, axis=-1, name="block1_batch1", training=training)
    features = tf.nn.relu(features, name="block1_relu1")
    features = tf.layers.conv2d(features, filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block1_conv2")
    features = tf.layers.batch_normalization(features, axis=-1, name="block1_batch2", training=training)
    features = tf.nn.relu(features, name="block1_relu2")
    features = tf.layers.max_pooling2d(features, pool_size=(2, 2), strides=(2, 2), name="block1_maxPool1")
    
    # Block 2
    features = tf.layers.conv2d(features, filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block2_conv1")
    features = tf.layers.batch_normalization(features, axis=-1, name="block2_batch1", training=training)
    features = tf.nn.relu(features, name="block2_relu1")
    features = tf.layers.conv2d(features, filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block2_conv2")
    features = tf.layers.batch_normalization(features, axis=-1, name="block2_batch2", training=training)
    features = tf.nn.relu(features, name="block2_relu2")
    features = tf.layers.max_pooling2d(features, pool_size=(2, 2), strides=(2, 2), name="block2_maxPool1")
    
    # Block 3
    features = tf.layers.conv2d(features, filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block3_conv1")
    features = tf.layers.batch_normalization(features, axis=-1, name="block3_batch1", training=training)
    features = tf.nn.relu(features, name="block3_relu1")
    features = tf.layers.conv2d(features, filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block3_conv2")
    features = tf.layers.batch_normalization(features, axis=-1, name="block3_batch2", training=training)
    features = tf.nn.relu(features, name="block3_relu2")
    features = tf.layers.conv2d(features, filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block3_conv3")
    features = tf.layers.batch_normalization(features, axis=-1, name="block3_batch3", training=training)
    features = tf.nn.relu(features, name="block3_relu3")
    features = tf.layers.max_pooling2d(features, pool_size=(2, 2), strides=(2, 2), name="block3_maxPool1")
    
    # Block 4
    features = tf.layers.conv2d(features, filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block4_conv1")
    features = tf.layers.batch_normalization(features, axis=-1, name="block4_batch1", training=training)
    features = tf.nn.relu(features, name="block4_relu1")
    features = tf.layers.conv2d(features, filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block4_conv2")
    features = tf.layers.batch_normalization(features, axis=-1, name="block4_batch2", training=training)
    features = tf.nn.relu(features, name="block4_relu2")
    features = tf.layers.conv2d(features, filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block4_conv3")
    features = tf.layers.batch_normalization(features, axis=-1, name="block4_batch3", training=training)
    features = tf.nn.relu(features, name="block4_relu3")
    features = tf.layers.max_pooling2d(features, pool_size=(2, 2), strides=(2, 2), name="block4_maxPool1")
    
    features = tf.layers.flatten(features)
    
    # Fully Connected
    features = tf.layers.dense(features, units=64, activation=tf.nn.relu, name="dense1")
    features = tf.layers.dropout(features, rate=0.5, name="dropout1")
    features = tf.layers.dense(features, units=32, activation=tf.nn.relu, name="dense2")
    features = tf.layers.dropout(features, rate=0.5, name="dropout2")
    logits = tf.layers.dense(features, units=NUM_LABELS, name="output")
    
    return logits

## Model Function for tf.Estimator
To save model for tensorflow serving, set **`export_outputs`** parameter in prediction mode

In [10]:
def model_fn(features, labels, mode):
    
    # to save model for tensorflow serving
    if isinstance(features, dict):
        features = features['image']
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        training = True
    else:
        training = False
    
    logits = model_structure(features, training)
    
    predicted_class = tf.argmax(logits, axis=1)
    
    # Prediction mode for tensorflow serving
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            "class_ids": predicted_class[:, tf.newaxis],
            "probability": tf.nn.softmax(logits),
            "logits": logits
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs={ 
            'classify': tf.estimator.export.PredictOutput(predictions)})
    
    # calculate cross entropy loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    def _f1_score(labels, predictions, class_id):
        """
        Reference: https://stackoverflow.com/questions/45603956/class-wise-precision-and-recall-for-multi-class-classification-in-tensorflow
        """
        precision = tf.metrics.precision_at_k(labels, predictions, 1, class_id)
        recall = tf.metrics.recall_at_k(labels, predictions, 1, class_id)
        f1_score = 2 * (precision[0] * recall[0]) / (precision[0] + recall[0])
        f1_score_op = 2 * (precision[1] * recall[1]) / (precision[1] + recall[1])
        return (f1_score, f1_score_op)
    
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predicted_class, name="accuracy")
    f1_score_BlackGrass = _f1_score(labels=labels, predictions=logits, class_id=0)
    metrics = {"accuracy" : accuracy, "f1_score_BlackGrass" : f1_score_BlackGrass}
    tf.summary.scalar("accuracy", accuracy[1])
    tf.summary.scalar("f1_score_BlackGrass", f1_score_BlackGrass[1])
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        
        # for multiple GPUs
        # optimizer = tf.contrib.estimator.TowerOptimizer(optimizer)
        
        # for batch normalization, tell tensorflow update batch normalization mean and variance
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
            
        # normal version
        #train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        
        # monitor trianing information
        logging_hook = tf.train.LoggingTensorHook({"loss" : loss, 
                                                   "accuracy" : accuracy[1], 
                                                   "f1_score_BlackGrass" : f1_score_BlackGrass[1]}, 
                                                  every_n_iter=SAVE_SUMMARY_STEPS)
        
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op, training_hooks=[logging_hook])
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

## For Multiple GPU (Parallel Computing)
Testing, not stable version  
Evaluation is not yet distributed

In [11]:
def get_distribution_strategy(num_gpus, all_reduce_alg=None):
    """Return a DistributionStrategy for running the model.
    Args:
    num_gpus: Number of GPUs to run this model.
    all_reduce_alg: Specify which algorithm to use when performing all-reduce.
      See tf.contrib.distribute.AllReduceCrossTowerOps for available algorithms.
      If None, DistributionStrategy will choose based on device topology.
    Returns:
    tf.contrib.distribute.DistibutionStrategy object.
    """
    if num_gpus == 0:
        return tf.contrib.distribute.OneDeviceStrategy("device:CPU:0")
    elif num_gpus == 1:
        return tf.contrib.distribute.OneDeviceStrategy("device:GPU:0")
    else:
        if all_reduce_alg:
            return tf.contrib.distribute.MirroredStrategy(
                num_gpus=num_gpus,
                cross_tower_ops=tf.contrib.distribute.AllReduceCrossTowerOps(
                    all_reduce_alg, num_packs=num_gpus))
        else:
            return tf.contrib.distribute.MirroredStrategy(num_gpus=num_gpus)

## Set Runing Config

In [12]:
session_config = tf.ConfigProto()
session_config.gpu_options.per_process_gpu_memory_fraction = 0.8
session_config.gpu_options.allow_growth = True
config = tf.estimator.RunConfig(model_dir=model_dir, 
                                tf_random_seed=seed, 
                                save_summary_steps=SAVE_SUMMARY_STEPS, 
                                save_checkpoints_secs=300, 
                                session_config=session_config,
                                keep_checkpoint_max=5, 
                                log_step_count_steps=100, )
#                                train_distribute=get_distribution_strategy(NUM_GPUS)) #for mutiple GPUs
clf = tf.estimator.Estimator(model_fn=model_fn, model_dir=model_dir, config=config)

INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_task_id': 0, '_num_ps_replicas': 0, '_model_dir': 'model_test/', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_is_chief': True, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_task_type': 'worker', '_service': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.8
  allow_growth: true
}
, '_master': '', '_tf_random_seed': 2, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbba6158198>, '_save_checkpoints_secs': 300, '_save_summary_steps': 100}


## Train and Evaluate

In [13]:
train_spec = tf.estimator.TrainSpec(lambda:train_input_fn_keras(generator), max_steps=1800)
eval_spec = tf.estimator.EvalSpec(lambda:eval_input_fn(valid_dataset, valid_labels), throttle_secs=300)
tf.estimator.train_and_evaluate(clf, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
output feature: (?, 128, 128, 3)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_test/model.ckpt.
INFO:tensorflow:loss = 2.8449872, step = 0
INFO:tensorflow:accuracy = 0.09765625, f1_score_BlackGrass = nan, loss = 2.8449872
INFO:tensorflow:global_step/sec: 0.984719
INFO:tensorflow:loss = 1.4894128, step = 100 (101.554 sec)
INFO:tensorflow:accuracy = 0.32421875, f1_score_BlackGrass = 0.1739130434782609, loss = 1.4894128 (101.554 sec)
INFO:tensorflow:global_step/

({'accuracy': 0.925,
  'f1_score_BlackGrass': 0.9,
  'global_step': 1800,
  'loss': 0.40996316},
 [])

## Save model for Tensorflow Serving
Can not save model after **predict**, because `Graph` is finalized and cannot be modified  
You can assign which model to be saved by `checkpoint_path` parameter

In [14]:
# input
image = tf.placeholder(tf.float32, shape=[None, 128, 128, 3], name='image')
# input receiver
input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
    'image': image,
})

clf.export_savedmodel("saved_model/", input_fn, checkpoint_path="model/model.ckpt-3600")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default', 'classify']
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from model/model.ckpt-3600
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model/temp-b'1536729364'/saved_model.pb


b'saved_model/1536729364'

## Load model and predict
Estimator predict method return **generator** type, so if you want to get all predictions please use for loop  
```python
for result in results:
    print(result)
```

In [15]:
results = clf.predict(lambda: eval_input_fn(valid_dataset, valid_labels), checkpoint_path="model/model.ckpt-3600")
next(results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-3600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'class_ids': array([8]),
 'logits': array([ 6.0683594 ,  2.4238575 , -0.16211988,  8.492256  ,  5.0945616 ,
        -5.9899054 , -2.91537   ,  2.8907135 , 28.753607  , 12.423214  ,
        -1.8675876 , 11.166189  ], dtype=float32),
 'probability': array([1.4057955e-10, 3.6739630e-12, 2.7673176e-13, 1.5871104e-09,
        5.3089269e-11, 8.1486327e-16, 1.7633500e-14, 5.8598668e-12,
        9.9999988e-01, 8.0872425e-08, 5.0278690e-14, 2.3008139e-08],
       dtype=float32)}

## Load model from `Estimator.export_savedmodel`
Reference: https://qiita.com/parkkiung123/items/13adb482860f356f97f3  

In [16]:
import tensorflow as tf
import numpy as np

export_dir = 'saved_model/1534315492'

with tf.Session(graph=tf.Graph()) as sess:
    # saved_model load
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir)
    # input
    i = sess.graph.get_tensor_by_name("image:0")
    # output
    r = sess.graph.get_tensor_by_name("ArgMax:0")
    print(sess.run(r, feed_dict={i:valid_dataset}))

INFO:tensorflow:Restoring parameters from saved_model/1534315492/variables/variables
[ 8  8  8  8  8  8  8  8  8  8  4 11  5 11 11 11 11 11 11 11  4  4  4  4
  4  4  4  4  4  4  1  1  1  1  1  1  1  1  1  1 10 10 10 10 10 10 10 10
 10 10  6  6  0  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  9  9
  9  9  9  8  9  9  9  9  3  3  3  3  3  3  3  3  3  3  5  5  5  5  5  5
  5  5  5  5  2  2  2  3  2  2  2  2  2  2  6  0  0  0  6  0  0  0  0  0]


## CNN Visualization
Reference: https://github.com/InFoCusp/tf_cnnvis  
Although you put multiple images, it only draws one image for visulization

In [17]:
import time
from tf_cnnvis import tf_cnnvis

# activation visualization
layers = ['r', 'p', 'c']

start = time.time()
image = sess.graph.get_tensor_by_name("image:0")
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir)
    # with sess_graph_path = None, the default Session will be used for visualization.
    is_success = tf_cnnvis.activation_visualization(sess_graph_path = None, value_feed_dict = {image : valid_dataset[10:11]}, 
                                                    layers=layers, path_logdir=os.path.join("Log","VGGNet"), 
                                                    path_outdir=os.path.join("Output","VGGNet"))
start = time.time() - start
print("Total Time = %f" % (start))

INFO:tensorflow:Restoring parameters from saved_model/1534315492/variables/variables
INFO:tensorflow:Restoring parameters from model/tmp-model
Reconstruction Completed for block1_relu1 layer. Time taken = 0.104183 s
Reconstruction Completed for block1_relu2 layer. Time taken = 0.113820 s
Reconstruction Completed for block2_relu1 layer. Time taken = 0.072773 s
Reconstruction Completed for block2_relu2 layer. Time taken = 0.078660 s
Reconstruction Completed for block3_relu1 layer. Time taken = 0.077696 s
Reconstruction Completed for block3_relu2 layer. Time taken = 0.081397 s
Reconstruction Completed for block3_relu3 layer. Time taken = 0.062347 s
Reconstruction Completed for block4_relu1 layer. Time taken = 0.059437 s
Reconstruction Completed for block4_relu2 layer. Time taken = 0.056464 s
Reconstruction Completed for block4_relu3 layer. Time taken = 0.055136 s
Reconstruction Completed for dense1/Relu layer. Time taken = 0.052904 s
Reconstruction Completed for dense2/Relu layer. Time ta

In [18]:
import time
from tf_cnnvis import tf_cnnvis

# deconv visualization
layers = ['r', 'p', 'c']

start = time.time()
image = sess.graph.get_tensor_by_name("image:0")
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir)
    is_success = tf_cnnvis.deconv_visualization(sess_graph_path = None, value_feed_dict = {image : valid_dataset[10:11]}, 
                                                layers=layers, path_logdir=os.path.join("Log","VGGNet"), 
                                                path_outdir=os.path.join("Output","VGGNet"))
start = time.time() - start
print("Total Time = %f" % (start))

INFO:tensorflow:Restoring parameters from saved_model/1534315492/variables/variables
INFO:tensorflow:Restoring parameters from model/tmp-model
Reconstruction Completed for block1_relu1 layer. Time taken = 0.466542 s
Reconstruction Completed for block1_relu2 layer. Time taken = 0.620932 s
Reconstruction Completed for block2_relu1 layer. Time taken = 1.099149 s
Reconstruction Completed for block2_relu2 layer. Time taken = 1.487387 s
Reconstruction Completed for block3_relu1 layer. Time taken = 2.165719 s
Reconstruction Completed for block3_relu2 layer. Time taken = 2.844257 s
Reconstruction Completed for block3_relu3 layer. Time taken = 2.891150 s
Reconstruction Completed for block4_relu1 layer. Time taken = 3.399595 s
Reconstruction Completed for block4_relu2 layer. Time taken = 3.881448 s
Reconstruction Completed for block4_relu3 layer. Time taken = 4.164734 s
Reconstruction Completed for dense1/Relu layer. Time taken = 3.040706 s
Reconstruction Completed for dense2/Relu layer. Time ta